In [22]:
from data_extraction import extract_data
import numpy as np
import pandas as pd

In [2]:
df = extract_data("mass_spectra_2023-07-20.msp")


In [3]:
from feature_generation import generate_features
from data_preparation import prepare_data

In [4]:
X_spectra,y = prepare_data(df)

In [5]:
X = X_spectra.apply(generate_features, axis=1).fillna(0)

In [44]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators = 250)

In [7]:
from sklearn.model_selection import cross_val_score
print(cross_val_score(clf, X,y, cv=5)) # [0.90326168 0.90400297 0.92105263 0.92364715 0.91178651] for initial features

[0.92550037 0.91882876 0.94662713 0.92883617 0.92661231]


In [125]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_full, y, test_size=0.1, random_state=42)

In [27]:
from confusion import plot_confusion_matrix

In [45]:
plot_confusion_matrix(clf, X_train, X_test, y_train, y_test)

,predicted 0,predicted 1,predicted 2,predicted 3
actual 0,943,1,0,0
actual 1,19,183,2,0
actual 2,2,26,82,9
actual 3,1,1,15,65


0 precision: 99.89406779661016
0 recall: 97.72020725388602
1 precision: 89.70588235294117
1 recall: 86.7298578199052
2 precision: 68.90756302521008
2 recall: 82.82828282828282
3 precision: 79.26829268292683
3 recall: 87.83783783783784


In [43]:
plot_confusion_matrix(clf_boost, X_train, X_test, y_train, y_test) #generated features

,predicted 0,predicted 1,predicted 2,predicted 3
actual 0,942,2,0,0
actual 1,6,186,12,0
actual 2,0,16,96,7
actual 3,0,0,6,76


0 precision: 99.78813559322035
0 recall: 99.36708860759494
1 precision: 91.17647058823529
1 recall: 91.17647058823529
2 precision: 80.67226890756302
2 recall: 84.21052631578947
3 precision: 92.6829268292683
3 recall: 91.56626506024097


In [31]:
from xgboost import XGBClassifier
clf_boost = XGBClassifier()

In [60]:
boost = np.array(sorted(list(zip(X.columns,clf_boost.feature_importances_)), key = lambda x: -x[1]))[:100,0]

In [61]:
forest = np.array(sorted(list(zip(X.columns,clf.feature_importances_)), key = lambda x: -x[1]))[:100,0]

In [68]:
plot_confusion_matrix(clf_boost, X_train, X_test, y_train, y_test) #initial features

,predicted 0,predicted 1,predicted 2,predicted 3
actual 0,939,5,0,0
actual 1,5,192,6,1
actual 2,0,16,96,7
actual 3,1,0,6,75


0 precision: 99.47033898305084
0 recall: 99.36507936507937
1 precision: 94.11764705882352
1 recall: 90.14084507042254
2 precision: 80.67226890756302
2 recall: 88.88888888888889
3 precision: 91.46341463414635
3 recall: 90.36144578313254


In [126]:
plot_confusion_matrix(clf_boost, X_train, X_test, y_train, y_test) #full features

,predicted 0,predicted 1,predicted 2,predicted 3
actual 0,940,4,0,0
actual 1,6,189,9,0
actual 2,0,11,100,8
actual 3,0,0,3,79


0 precision: 99.57627118644068
0 recall: 99.36575052854123
1 precision: 92.64705882352942
1 recall: 92.64705882352942
2 precision: 84.03361344537815
2 recall: 89.28571428571429
3 precision: 96.34146341463415
3 recall: 90.80459770114942


In [70]:
from sklearn.feature_selection import RFE

In [71]:
selector = RFE(clf_boost, n_features_to_select=100, step=5)

In [72]:
selector.fit(X_train, y_train)

RFE(estimator=XGBClassifier(base_score=None, booster=None, callbacks=None,
                            colsample_bylevel=None, colsample_bynode=None,
                            colsample_bytree=None, early_stopping_rounds=None,
                            enable_categorical=False, eval_metric=None,
                            feature_types=None, gamma=None, gpu_id=None,
                            grow_policy=None, importance_type=None,
                            interaction_constraints=None, learning_rate=None,
                            max_bin=None, max_cat_threshold=None,
                            max_cat_to_onehot=None, max_delta_step=None,
                            max_depth=None, max_leaves=None,
                            min_child_weight=None, missing=nan,
                            monotone_constraints=None, n_estimators=100,
                            n_jobs=None, num_parallel_tree=None,
                            objective='multi:softprob', predictor=None, ...),
    n_features_to_select=100, step=5)

In [73]:
selector

RFE(estimator=XGBClassifier(base_score=None, booster=None, callbacks=None,
                            colsample_bylevel=None, colsample_bynode=None,
                            colsample_bytree=None, early_stopping_rounds=None,
                            enable_categorical=False, eval_metric=None,
                            feature_types=None, gamma=None, gpu_id=None,
                            grow_policy=None, importance_type=None,
                            interaction_constraints=None, learning_rate=None,
                            max_bin=None, max_cat_threshold=None,
                            max_cat_to_onehot=None, max_delta_step=None,
                            max_depth=None, max_leaves=None,
                            min_child_weight=None, missing=nan,
                            monotone_constraints=None, n_estimators=100,
                            n_jobs=None, num_parallel_tree=None,
                            objective='multi:softprob', predictor=None, ...),
    n_features_to_select=100, step=5)

In [74]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
LDA = LinearDiscriminantAnalysis()

In [77]:
plot_confusion_matrix(LDA, X_train, X_test, y_train, y_test)

,predicted 0,predicted 1,predicted 2,predicted 3
actual 0,930,9,1,4
actual 1,12,177,12,3
actual 2,0,41,61,17
actual 3,2,2,26,52


0 precision: 98.51694915254238
0 recall: 98.51694915254238
1 precision: 86.76470588235294
1 recall: 77.29257641921397
2 precision: 51.26050420168067
2 recall: 61.0
3 precision: 63.41463414634146
3 recall: 68.42105263157895


In [78]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [80]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
pca = PCA()
X_train_pca = pca.fit_transform(X_train_scaled)

In [82]:
X_test_pca = pca.transform(scaler.transform(X_test))

In [111]:
n_features = 2163
plot_confusion_matrix(LDA, X_train_pca[:, :n_features], X_test_pca[:, :n_features], y_train, y_test)

,predicted 0,predicted 1,predicted 2,predicted 3
actual 0,929,8,6,1
actual 1,9,176,13,6
actual 2,0,40,57,22
actual 3,4,4,24,50


0 precision: 98.41101694915254
0 recall: 98.61995753715499
1 precision: 86.27450980392157
1 recall: 77.19298245614034
2 precision: 47.89915966386555
2 recall: 56.99999999999999
3 precision: 60.97560975609756
3 recall: 63.29113924050633


In [98]:
len(pca.explained_variance_ratio_)

2163

In [114]:
X

,AC-14,BASE,CENT,EVEN,LG1-100,LG1-1000,LG1-1001,LG1-1002,LG1-1003,LG1-1004,...,MOD14-14,MOD14-2,MOD14-3,MOD14-4,MOD14-5,MOD14-6,MOD14-7,MOD14-8,MOD14-9,SYMX
0,7.156032,20.169594,44.867039,36.624268,-0.001001,0.0,0.0,0.0,0.0,0.0,...,0.074074,0.122855,0.919603,0.161698,0.403794,0.080397,0.455285,1.000000,0.471545,79.333333
1,13.056750,9.428086,47.975376,38.089845,-2.389763,0.0,0.0,0.0,0.0,0.0,...,0.263294,0.263294,1.000000,0.360351,0.523490,0.174497,0.411977,0.733092,0.535880,74.375000
2,23.008660,7.562453,57.926794,53.035579,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.621652,0.928571,0.848214,0.300781,0.171875,0.305804,0.297991,0.786272,0.290179,57.202505
3,25.332300,7.857480,61.731520,50.747208,-0.183322,0.0,0.0,0.0,0.0,0.0,...,0.167416,0.397301,0.675162,0.398301,0.419790,0.720140,0.346327,1.000000,0.725137,98.898072
4,14.075792,15.663217,30.321560,17.842584,-0.183322,0.0,0.0,0.0,0.0,0.0,...,0.131772,0.107613,1.000000,0.163250,0.911420,0.112006,0.426061,0.150805,0.432650,19.518717
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13485,0.469149,37.542277,68.638883,17.549793,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.130293,0.014984,0.008469,0.021498,0.262541,0.069055,0.020195,0.050814,0.108795,83.690987
13486,0.070561,56.092083,87.207894,19.427288,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.129969,0.005352,0.003058,0.008410,0.013761,0.013761,0.030581,0.028287,0.023700,98.984772
13487,11.776692,33.057578,65.650399,38.583719,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.203731,0.037313,0.027612,0.020896,0.028358,0.020896,0.026866,0.023134,0.047015,73.033708
13488,3.656095,42.294666,90.380474,33.403895,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.164041,0.012975,0.016682,0.016682,0.040778,0.029657,0.030584,0.039852,0.120482,98.477157


In [124]:
X_full = X.join(X_spectra)

In [128]:
from sklearn.feature_selection import SelectKBest
slc = SelectKBest()
X_full.columns = X_full.columns.astype(str)
slc.fit(X_full,y)
scores = slc.scores_

c:\Users\schne\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [   5    6    7    8    9   10   11   12   13   14   16   17   18   19
   20   21   22   23   24   25   27   28   29   30   31   32   33   34
   35   36   38   39   40   41   42   43   44   45   46   47   49   50
   51   52   53   54   55   56   57   58   60   61   62   63   64   65
   66   67   68   69   71   72   73   74   75   76   77   78   79   80
   82   83   84   85   86   87   88   89   90   91   93   94   95   96
   97   98   99  100  101  102  104  105  106  107  108  109  110  111
  112  113  115  116  117  118  119  120  121  122  123  124  126  127
  128  129  130  131  132  133  134  135  137  138  792  808  809  820
  857  858  876  877  894  897  905  906  907  917  918  919  920  921
  923  924  925  934  935  936  937  938  939  946  947  949  950  951
  952  953  954  956  957  958  959  962  963  964  973  974  975  9

In [153]:
features = sorted([x for x in list(zip(X_full.columns,scores)) if x[1]>0], key = lambda x: -x[1])

In [159]:
features

[('LG1-72', 9764.703954812923),
 ('LG2-71', 9638.405338348603),
 ('73', 8295.02486583122),
 ('LG1-73', 6367.086360617022),
 ('LG1-75', 5476.951121945591),
 ('MOD14-3', 2724.3826437204148),
 ('LG2-75', 2691.871602622755),
 ('LG1-146', 2164.490218494866),
 ('LG2-145', 2078.9824197889425),
 ('LG2-73', 1940.9263718043546),
 ('LG2-74', 1799.8426913904777),
 ('LG1-147', 1563.815487213435),
 ('75', 1452.2294395659844),
 ('LG2-147', 1220.8836915533138),
 ('LG2-72', 958.297381811347),
 ('147', 909.3161937601824),
 ('LG1-74', 903.8807325058048),
 ('LG1-59', 860.7995465588778),
 ('LG2-57', 747.9077167259526),
 ('LG1-133', 658.7231228856308),
 ('LG1-216', 588.5023248904299),
 ('LG1-58', 542.4360512704894),
 ('MOD14-5', 480.9305679504515),
 ('LG2-215', 476.46325577950756),
 ('LG2-133', 435.5829946408971),
 ('LG2-148', 408.8738098756627),
 ('LG2-127', 407.49939395475013),
 ('LG1-128', 403.78606137325806),
 ('LG1-71', 403.3876695245525),
 ('CENT', 396.7744979954837),
 ('LG1-149', 367.93087647208546),